In [1]:
import json
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
jsonFile = '2019/2019schedule.json'

In [3]:
curSch = pd.read_json(jsonFile)
# curSch = curSch[['week','away_team','away_points','home_points','home_team',
#                  'away_conference','home_conference','neutral_site']]

In [4]:
curSch.loc[curSch.away_team == 'Notre Dame','away_conference'] = 'Notre Dame'
curSch.loc[curSch.home_team == 'Notre Dame','home_conference'] = 'Notre Dame'

In [5]:
FBSteams = curSch['home_team'].unique()
FBSteams.sort()

In [6]:
P5 = ['ACC','Pac-12','Big Ten','Big 12','SEC','Notre Dame']
G5 = ['Conference USA','Sun Belt','Mid-American','American Athletic','Mountain West','FBS Independents']

In [7]:
FBSdict = {}
for team in FBSteams:
    FBSdict[team] = curSch[(curSch['away_team'] == team) | 
                           (curSch['home_team'] == team)]
    FBSdict[team] = FBSdict[team][np.isfinite(FBSdict[team].away_points)]
    FBSdict[team] = FBSdict[team].reset_index()

In [8]:
def seasonProgression(dfR):
    totalGames = len(dfR)
    curWeek = int(dfR['week']) + 1
    return curWeek/(2*totalGames)

In [9]:
def winningTeam(dfR):
    if dfR['home_points'] > dfR['away_points']:
        return dfR['home_team']
    else:
        return dfR['away_team']
    
def teamWin(dfR, team):
    if dfR.winner == team:
        return True
    else:
        return False

In [10]:
def findOpp(dfR):
    if dfR.home == True:
        return dfR.away_team
    else:
        return dfR.home_team
def findOppConf(dfR):
    if dfR.home == True:
        return dfR.away_conference
    else:
        return dfR.home_conference

In [11]:
def locMult(dfR):
    if dfR['neutral_site'] == True:
        return 1.05
    elif dfR['home'] == True:
        return 1.1
    else:
        return 1.

In [12]:
def pointDiff(dfR,team):
    if dfR.home_team == team:
        return dfR.home_points - dfR.away_points
    else:
        return dfR.away_points - dfR.home_points

In [13]:
def confMult(dfR):
    if dfR.opp_conference in P5:
        return 1.
    elif dfR.opp_conference in G5:
        return 0.5
    else:
        return 0.15

In [14]:
def winQ(dfR,week): 
    if dfR.opp_conference == None:
        oppWinPct = 0.5
    else:
#         print(dfR.opp_conference)
        oppWinPct = FBSdict[dfR.opp].loc[FBSdict[dfR.opp].week <= week, 'winPct'].tail(1).item()
    
    if dfR.teamWin == True:
        winMult = 1
        oppFactor = oppWinPct
    else:
        winMult = -1
        oppFactor = 1 - oppWinPct
    
    return oppFactor * dfR.seasonProg * dfR.conf_mult * winMult*(dfR.loc_mult + 0.01*dfR.point_diff)

***


In [15]:
FBSdict['Rutgers']['seasonProg'] = FBSdict['Rutgers'].apply(lambda row: seasonProgression(row), axis = 1)
FBSdict['Rutgers']['winner'] = FBSdict['Rutgers'].apply(lambda row: winningTeam(row), axis = 1)

In [16]:
wins = 0
losses = 0
winList = []
lossList = []
for row in FBSdict['Rutgers'].itertuples():
#     print(row.winner)
    if row.winner == 'Rutgers':
        wins += 1
    else:
        losses += 1
    winList.append(wins)
    lossList.append(losses)

In [17]:
winList, lossList
FBSdict['Rutgers']['wins'] = winList
FBSdict['Rutgers']['losses'] = lossList

In [18]:
hometeam = FBSdict['Rutgers']['home_team'] == 'Rutgers'
FBSdict['Rutgers']['home'] = hometeam

In [19]:
FBSdict['Rutgers']['winPct'] = FBSdict['Rutgers']['wins']/(FBSdict['Rutgers']['losses'] + FBSdict['Rutgers']['wins'])

In [20]:
def findOpp(dfR):
    if dfR.home == True:
        return dfR.away_team
    else:
        return dfR.home_team
def findOppConf(dfR):
    if dfR.home == True:
        return dfR.away_conference
    else:
        return dfR.home_conference

In [21]:
FBSdict['Rutgers']['opp'] = FBSdict['Rutgers'].apply(lambda row: findOpp(row), axis = 1)
FBSdict['Rutgers']['opp_conference'] = FBSdict['Rutgers'].apply(lambda row: findOppConf(row), axis = 1)

In [22]:
def locMult(dfR):
    if dfR['neutral_site'] == True:
        return 1.05
    elif dfR['home'] == True:
        return 1.1
    else:
        return 1.

In [23]:
def pointDiff(dfR,team):
    if dfR.home_team == team:
        return dfR.home_points - dfR.away_points
    else:
        return dfR.away_points - dfR.home_points

In [24]:
FBSdict['Rutgers']['point_diff'] = FBSdict['Rutgers'].apply(lambda row: pointDiff(row,'Rutgers'), axis = 1)

In [25]:
FBSdict['Rutgers']['loc_mult'] = FBSdict['Rutgers'].apply(lambda row: locMult(row), axis = 1)

In [26]:
def confMult(dfR):
    if dfR.opp_conference in P5:
        return 1.
    elif dfR.opp_conference in G5:
        return 0.5
    else:
        return 0.15

In [37]:
FBSdict['Rutgers']['conf_mult'] = FBSdict['Rutgers'].apply(lambda row: confMult(row), axis = 1)

In [38]:
FBSdict['Rutgers'].winPct

0     1.000000
1     0.500000
2     0.333333
3     0.250000
4     0.200000
5     0.166667
6     0.142857
7     0.250000
8     0.222222
9     0.200000
10    0.181818
11    0.166667
Name: winPct, dtype: float64

In [39]:
FBSdict['Rutgers'].loc[FBSdict['Rutgers'].week <= 11, 'winPct'].tail(1).iloc[0]

0.2222222222222222

In [40]:
FBSdict['Rutgers']

,index,id,season,week,season_type,start_date,neutral_site,conference_game,attendance,venue_id,...,winner,wins,losses,home,winPct,opp,opp_conference,point_diff,loc_mult,conf_mult
0,20,401112264,2019,1,regular,2019-08-30T23:15:00.000Z,False,False,NaN,3754,...,Rutgers,1,0,True,1.000000,UMass,FBS Independents,27.0,1.1,0.5
1,91,401112192,2019,2,regular,2019-09-07T16:00:00.000Z,False,True,NaN,3793,...,Iowa,1,1,False,0.500000,Iowa,Big Ten,-30.0,1.0,1.0
2,234,401112265,2019,4,regular,2019-09-21T16:00:00.000Z,False,False,NaN,3754,...,Boston College,1,2,True,0.333333,Boston College,ACC,-14.0,1.1,1.0
3,299,401112225,2019,5,regular,2019-09-28T16:00:00.000Z,False,True,NaN,3558,...,Michigan,1,3,False,0.250000,Michigan,Big Ten,-52.0,1.0,1.0
4,346,401112205,2019,6,regular,2019-10-05T16:00:00.000Z,False,True,NaN,3754,...,Maryland,1,4,True,0.200000,Maryland,Big Ten,-41.0,1.1,1.0
5,394,401112162,2019,7,regular,2019-10-12T16:00:00.000Z,False,True,NaN,3830,...,Indiana,1,5,False,0.166667,Indiana,Big Ten,-35.0,1.0,1.0
6,476,401112234,2019,8,regular,2019-10-19T19:30:00.000Z,False,True,NaN,3754,...,Minnesota,1,6,True,0.142857,Minnesota,Big Ten,-35.0,1.1,1.0
7,505,401112266,2019,9,regular,2019-10-26T16:00:00.000Z,False,False,NaN,3754,...,Rutgers,2,6,True,0.250000,Liberty,FBS Independents,10.0,1.1,0.5
8,579,401112153,2019,10,regular,2019-11-02T19:30:00.000Z,False,True,NaN,3832,...,Illinois,2,7,False,0.222222,Illinois,Big Ten,-28.0,1.0,1.0
9,682,401112255,2019,12,regular,2019-11-16T20:30:00.000Z,False,True,NaN,3754,...,Ohio State,2,8,True,0.200000,Ohio State,Big Ten,-35.0,1.1,1.0


In [44]:
FBSdict['Rutgers']['win_q'] = FBSdict['Rutgers'].apply(lambda row: winQ(row,4),axis = 1)

KeyError: ('winPct', 'occurred at index 0')

In [33]:
FBSdict['Alabama']

,index,id,season,week,season_type,start_date,neutral_site,conference_game,attendance,venue_id,...,home_team,home_conference,home_points,home_line_scores,home_post_win_prob,away_team,away_conference,away_points,away_line_scores,away_post_win_prob
0,44,401110720,2019,1,regular,2019-08-31T19:30:00.000Z,True,False,NaN,5348,...,Alabama,SEC,42.0,"[0, 14, 21, 7]",0.999991,Duke,ACC,3.0,"[0, 3, 0, 0]",0.000009
1,118,401110773,2019,2,regular,2019-09-07T20:00:00.000Z,False,False,NaN,3657,...,Alabama,SEC,62.0,"[21, 17, 24, 0]",0.999994,New Mexico State,FBS Independents,10.0,"[0, 0, 7, 3]",0.000006
2,186,401110794,2019,3,regular,2019-09-14T19:30:00.000Z,False,True,NaN,3994,...,South Carolina,SEC,23.0,"[10, 0, 3, 10]",0.069151,Alabama,SEC,47.0,"[14, 10, 10, 13]",0.930849
3,242,401110797,2019,4,regular,2019-09-21T16:00:00.000Z,False,False,NaN,3657,...,Alabama,SEC,49.0,"[14, 14, 14, 7]",0.999986,Southern Mississippi,Conference USA,7.0,"[0, 7, 0, 0]",0.000014
4,305,401110806,2019,5,regular,2019-09-28T19:30:00.000Z,False,True,NaN,3657,...,Alabama,SEC,59.0,"[7, 31, 14, 7]",0.999813,Ole Miss,SEC,31.0,"[10, 0, 7, 14]",0.000187
5,412,401110822,2019,7,regular,2019-10-12T19:30:00.000Z,False,True,NaN,3795,...,Texas A&M,SEC,28.0,"[7, 6, 7, 8]",0.013617,Alabama,SEC,47.0,"[14, 10, 10, 13]",0.986383
6,498,401110824,2019,8,regular,2019-10-20T01:00:00.000Z,False,True,NaN,3657,...,Alabama,SEC,35.0,"[14, 7, 7, 7]",0.997619,Tennessee,SEC,13.0,"[7, 3, 3, 0]",0.002381
7,544,401110831,2019,9,regular,2019-10-26T23:00:00.000Z,False,True,NaN,3657,...,Alabama,SEC,48.0,"[17, 24, 7, 0]",0.999996,Arkansas,SEC,7.0,"[0, 0, 0, 7]",0.000004
8,635,401110842,2019,11,regular,2019-11-09T20:30:00.000Z,False,True,NaN,3657,...,Alabama,SEC,41.0,"[7, 6, 7, 21]",0.051029,LSU,SEC,46.0,"[10, 23, 0, 13]",0.948971
9,663,401110851,2019,12,regular,2019-11-16T17:00:00.000Z,False,True,NaN,3693,...,Mississippi State,SEC,7.0,"[7, 0, 0, 0]",0.003604,Alabama,SEC,38.0,"[21, 14, 3, 0]",0.996396


In [34]:
FBSdict['Rutgers'].win_q.mean()

AttributeError: 'DataFrame' object has no attribute 'win_q'

In [35]:
FBSdict['Rutgers'].tail(1).wins.item(),FBSdict['Rutgers'].tail(1).losses.item()

/home/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: `item` has been deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


(2, 10)